# Garch model experiment 01

In [2]:
import os
import sys

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from arch import arch_model
from tqdm import tqdm

sns.set_style("darkgrid")
plt.rc("figure", figsize=(16, 8))
plt.rc("font", size=13)

In [3]:
TICKER = "BTC-USD"
DATA_DIR = "data_hourly"
FILE_PATH = os.path.join("..", "data", f"{DATA_DIR}", f"{TICKER}_hourly.csv")

In [11]:
try:
    df_raw = pd.read_csv(
        FILE_PATH,
        header=0,           
        skiprows=[1, 2],           
        parse_dates=True    
    )
    print(f"Данные для {TICKER} успешно загружены.")
except FileNotFoundError:
    print(f"Ошибка: Файл не найден по пути {FILE_PATH}. Убедитесь, что вы запустили скрипт скачивания.")
    exit()

Данные для BTC-USD успешно загружены.


In [12]:
df = df_raw.rename(columns={'Price': 'Datetime'})

df['Datetime'] = pd.to_datetime(df['Datetime'])

df.set_index('Datetime', inplace=True)

In [14]:
print(df.shape)
df.head()

(17492, 5)


,Close,High,Low,Open,Volume
Datetime,,,,,
2023-09-17 21:00:00+00:00,26487.957031,26500.771484,26420.402344,26443.996094,0
2023-09-17 22:00:00+00:00,26467.375000,26491.326172,26450.060547,26491.326172,44983296
2023-09-17 23:00:00+00:00,26536.810547,26536.810547,26454.970703,26490.833984,71581184
2023-09-18 00:00:00+00:00,26478.710938,26555.085938,26415.515625,26532.994141,167976448
2023-09-18 01:00:00+00:00,26471.654297,26488.355469,26455.878906,26474.019531,0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17492 entries, 2023-09-17 21:00:00+00:00 to 2025-09-15 18:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   17492 non-null  float64
 1   High    17492 non-null  float64
 2   Low     17492 non-null  float64
 3   Open    17492 non-null  float64
 4   Volume  17492 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 819.9 KB


In [16]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Close,17492.0,7.404251e+04,2.609701e+04,26063.386719,57715.159180,68160.718750,9.730265e+04,1.238109e+05
High,17492.0,7.425237e+04,2.615442e+04,26102.777344,57951.115234,68365.589844,9.755880e+04,1.243223e+05
Low,17492.0,7.382119e+04,2.603935e+04,26011.468750,57517.906250,67933.402344,9.700106e+04,1.234140e+05
Open,17492.0,7.403691e+04,2.609704e+04,26059.931641,57714.226562,68167.683594,9.729354e+04,1.238693e+05
Volume,17492.0,4.355266e+08,1.127467e+09,0.000000,0.000000,0.000000,4.715523e+08,3.461036e+10
